<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">pyCRYPTO</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);"><b style=color:red;>Crypto</b>graphy</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
<td><img style="height: 150px;" src="images/pyCRYPTO.png"></td>
</tr>
</table>

----
# `pyCRYPTO`

pyCRYPTO, a program package for cryptography tools and blockchains.

# RSA cryption: real keys
----

In this notebook, we introduce and discuss the **RSA cryptographic scheme**, which has been
named from the initials of its inventors, Rivest–Shamir–Adleman.

In [1]:
import numpy as np
import libCrypt

The **RSA** cryptographic scheme is widely used for encrypting messages and signatures, using a
**public-private key** pair.

It is based on the fact, that some operations, such as **prime factorisation** of a larger
integer number, is very tedious, but recovering the number from **multiplication** of
the primes is easy.

This asymmetric behaviour is also known as **one-way function**: 
- Easy to calculate in the forward direction, 
- Difficult in the reverse direction.

To make use of the RSA method, we need to perform three steps:
- Generate a **private-public key pair**.
- **Encrypt** a message using the public key.
- **Decrypt** a message using the private key.

----
## Private-public key pair

[See also wikipedia...](https://en.wikipedia.org/wiki/RSA_(cryptosystem))

- Choose two large prime numbers, $p$ and $q$, with $0.1 < |\log p - \log q| < 30$
- Calculate RSA module $n=p q$
- Calculate Euler function $\Phi(n)=(p-1)(q-1)$
- Choose integer number $e$, which is coprime to both $p-1$ and $q-1$
- Calculate $d$ as modular inverse $d$, thus $e d \equiv 1 \mbox{ mod } \Phi(n)$ has to hold.

Then, we have obtained the **public-private key pair**:
- **public key:** $(n,e)$
- **private key:** $(n,d)$

In [2]:
def RSAcreateKeys(keySize=1024):
    """
    pyCRYPTO
    create public-private key sequence for RSA crypting
    input:
      keySize - key size 
    output:
      publicKey  - public key tuple
      privateKey - private key tuple
    """
    import random
    import libCrypt
    # generate two large prime
    p = libCrypt.generateLargePrime(keySize)
    q = libCrypt.generateLargePrime(keySize)
    # calculate product of primes
    n = p*q
    # calculate phi-function
    phi=(p-1)*(q-1)
    # search number e, which is coprime to phi
    while True:
        # Keep trying random numbers for e until one is valid.
        e = random.randrange(2**(keySize-1),2**(keySize))
        if libCrypt.greatestCommonDivisor(e, (p-1)*(q-1)) == 1:
            break
    # calculate d as modular inverse e mod phi
    d = pow(e,-1,phi)    
    #print(p,q,n,e,d)
    # save askey pair
    publicKey = (keySize,n,e)
    privateKey = (keySize,n,d)
    return publicKey,privateKey

In [3]:
keySize = 128
keySize = 1024
publicKey,privateKey = RSAcreateKeys(keySize)
print('public key:  ',publicKey)
print('private key: ',privateKey)

public key:   (1024, 12500255324603654360598271612205651921367399145160431137730066105425503058718876055880990308439873804748109888868000824440604413667315864574953029611343840596695480921032987162432205601381749648971988598945157583664645703244956589005237802221720937514249800512255962487128818373961745209594990413071485097886946019486913590236966399325272085467607254556259813223615479029294261722893452284125263566076069502279122264402934882898881925492399521926428210448973585857579982747382055015752583963322157319233303000149191411359118968029428808133194618093029054080605530689158168862956738878680021691262630237044323069200083, 168287967901377487796971759523690867091908144324006795543131699147313690581196854425774324360676749115073424168222318324622085860835130831495792932080070224771466839976925715855150983019510757045000782816344039113496995245746614339324584497757471934370882488903984214485150601782354949245413035000661408764637)
private key:  (1024, 1250025532460365436059827161

### Check
We check to private-public keys with a short message consisting of integer numbers:
$$
\begin{array}{rcl}
c &=& m^e \mbox{ mod } n \\
m &=& c^d \mbox{ mod } n
\end{array}
$$

In [4]:
# message
m=1234567890123456789
print('m: ',m)
# keys
keySize,n,e = publicKey
keySize,n,d = privateKey
# encrypt with forward relation
c=pow(m,e,n)
print('c: ',c)
# decrypt with backward relation
mback=pow(c,d,n)
print('mback: ',mback)

print('Check: ',m==mback)

m:  1234567890123456789
c:  1065615036925380900722179870103267372937979023490493393275215503667456635204940603547839816852851678037676473428153641433671205319864776284620021656450930725492962192767086437607337051075500517570097779278180051476682438298908146561852811816426746205683984779722028054218149228256578501436854618202220682924479506273473678790799224850790083752789900732971543587253468184467645853012360050858166333278651220998958627382472226473861029416124750398779869456180999921822908846078489676622029679614334641185454836065626526401859047218291428317795286916739244776229705509703042778094296204663997820932409899794631654563817
mback:  1234567890123456789
Check:  True


----
## Split message to bytes
Each message is splitted into bytes, then each byte in encrypted.

In [5]:
message = "Allwissend bin ich nicht; doch viel ist mir bewusst!"
L = len(message)
print(L,type(message),message)

52 <class 'str'> Allwissend bin ich nicht; doch viel ist mir bewusst!


In [6]:
# encode message string to bytes
messageBytes = message.encode('utf-8')
print(type(messageBytes),messageBytes)
# print each byte
for byte in messageBytes:
    print(byte,'=>',chr(byte),' ',end='')
print()
    
# decode message back
messageBack = messageBytes.decode()
print(type(messageBack),messageBack)

<class 'bytes'> b'Allwissend bin ich nicht; doch viel ist mir bewusst!'
65 => A  108 => l  108 => l  119 => w  105 => i  115 => s  115 => s  101 => e  110 => n  100 => d  32 =>    98 => b  105 => i  110 => n  32 =>    105 => i  99 => c  104 => h  32 =>    110 => n  105 => i  99 => c  104 => h  116 => t  59 => ;  32 =>    100 => d  111 => o  99 => c  104 => h  32 =>    118 => v  105 => i  101 => e  108 => l  32 =>    105 => i  115 => s  116 => t  32 =>    109 => m  105 => i  114 => r  32 =>    98 => b  101 => e  119 => w  117 => u  115 => s  115 => s  116 => t  33 => !  
<class 'str'> Allwissend bin ich nicht; doch viel ist mir bewusst!


----
## Encrypt

Each byte $m$ of a **message** will be encrypted to an encrypted message $c$, using the public key $(n,e)$:
$$
c = m^e \mbox{ mod } n
$$

In [8]:
encrypted_message = libCrypt.RSAencrypt(message,publicKey)
print(encrypted_message)

[...]


----
## Decrypt

A **crypted message** $c$ will be decrypted to a message $m$, using the private key:
$$
m = c^d \mbox{ mod } n
$$

In [9]:
decrypted_message = libCrypt.RSAdecrypt(encrypted_message,privateKey,debug=False)
print(decrypted_message)

Allwissend bin ich nicht; doch viel ist mir bewusst!


----
## Example with longer text


In [11]:
message = "Am farbigen Abglanz haben wir das Leben.\n"
message += "Allwissend bin ich nicht; doch viel ist mir bewusst!\n"
message += "Alt wird man wohl, wer aber klug?\n"
message += "Von Zeit zu Zeit seh'ich den Alten gern und hüte mich, mit ihm zu brechen.\n"
message += "Anmut bringen wir ins Leben. Leget Anmut in das Geben. Leget Anmut ins Empfangen, Lieblich ist's, den Wunsch erlangen.\n"
message += "Und in stiller Tage Schranken. Höchst anmutig sei das Danken.\n"
message += "Mein schönes Fräulein, darf ich wagen, Meinen Arm und Geleit Ihr anzutragen?\n"
message += "Was glänzt, ist für den Augenblick geboren.\n"
message += "Wo so ein Köpfchen keinen Ausgang sieht, stellt es sich gleich das Ende vor.\n"
message += "Hoch ist der Doppelgewinn zu schätzen, barmherzig sein und sich zugleich ergetzen.\n"
message += "Wer immer strebend sich bemüht, den können wir erlösen.\n"
message += "- Johann Wolfgang von Goethe"

print(' ===========','\n','=original==','\n','===========')
print(message)

encrypted_message = libCrypt.RSAencrypt(message,publicKey)
decrypted_message = libCrypt.RSAdecrypt(encrypted_message,privateKey)
print(' ===========','\n','=recovered=','\n','===========')
print(decrypted_message)

 =original== 
Am farbigen Abglanz haben wir das Leben.
Allwissend bin ich nicht; doch viel ist mir bewusst!
Alt wird man wohl, wer aber klug?
Von Zeit zu Zeit seh'ich den Alten gern und hüte mich, mit ihm zu brechen.
Anmut bringen wir ins Leben. Leget Anmut in das Geben. Leget Anmut ins Empfangen, Lieblich ist's, den Wunsch erlangen.
Und in stiller Tage Schranken. Höchst anmutig sei das Danken.
Mein schönes Fräulein, darf ich wagen, Meinen Arm und Geleit Ihr anzutragen?
Was glänzt, ist für den Augenblick geboren.
Wo so ein Köpfchen keinen Ausgang sieht, stellt es sich gleich das Ende vor.
Hoch ist der Doppelgewinn zu schätzen, barmherzig sein und sich zugleich ergetzen.
Wer immer strebend sich bemüht, den können wir erlösen.
- Johann Wolfgang von Goethe
 =recovered= 
Am farbigen Abglanz haben wir das Leben.
Allwissend bin ich nicht; doch viel ist mir bewusst!
Alt wird man wohl, wer aber klug?
Von Zeit zu Zeit seh'ich den Alten gern und hüte mich, mit ihm zu brechen.
Anmut bringen wir i

----